In [ ]:
import tensorflow
from tensorflow import keras

**Implementing ResNet-34 CNN**

In [ ]:
# to avoid repeating, metes la funcion y lo que quieres que sea siempre igual
from functools import partial
ResConv2D = partial(keras.layers.Conv2D, 
                    kernel_size = 3, 
                    padding = "same",
                    use_bias = False)
# al llamarla, le metes los otros argumentos

class ResidualUnit(keras.layers.Layer):
    def __init__(self, filters, strides = 1, activation = "relu", **kwargs):
        super().__init__(**kwargs)
        self.activation = keras.activations.get(activation) # to get the activation function
        # las del camino principal:
        self.main_layers = [
            ResConv2D(filters = filters, strides = strides),
            keras.layers.BatchNormalization(),
            self.activation,
            ResConv2D(filters = filters, strides = 1), # in this case strides = 1 always
            keras.layers.BatchNormalization(), # no se acaba en activation, eso lo hacemos despues con la suma de ambas
        ]
        # las de la skip connection:
        self.skip_layers = [] # de normal nada porque los tamaños van a ser =
        if strides > 1: # si los tamaños no son =, hay que reducir tambien el del skip connection
            self.skip_layers = [
                ResConv2D(filters = filters, kernel_size = 1, strides = strides),
                keras.layers.BatchNormalization()
            ]
            # una conv 1x1 con strides = strides para que las dims finales sean =

    def call(self, inputs):
        # por un lado el camino normal
        Z = inputs
        for layer in self.main_layers:
            Z = layer(Z)
        # por otro lado la skip connection
        skip_Z = inputs
        for layer in self.skip_layers:
            skip_Z = layer(skip_Z)
        # la suma de ambas
        Z_added = Z + skip_Z
        # activacion que no estaba en self.main_layers al final
        return self.activation(Z_added)

    # si quisieramos guardar hiperparams etc habria que hacer el get_config() ...

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(filters = 64, kernel_size = 7,
                              strides = 2, input_shape = [224, 224, 3],
                              padding = "same", use_bias = False))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation("relu"))
model.add(keras.layers.MaxPool2D(pool_size = 3, strides = 2, padding = "same"))

# if it is the first one after a change, strides = 2
# al duplicar el numero de filters, se reduce las dimensiones con el stride
prev_filters = 64
for filters in [64] * 3 + [128] * 4 + [256] * 6 + [512] * 3:
    strides = 1 if filters == prev_filters else 2
    model.add(ResidualUnit(filters, strides = strides))
    prev_filters = filters

model.add(keras.layers.GlobalAvgPool2D())
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(10, activation = "softmax"))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 112, 112, 64)      9408      
                                                                 
 batch_normalization (BatchN  (None, 112, 112, 64)     256       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 112, 112, 64)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 56, 56, 64)       0         
 )                                                               
                                                                 
 residual_unit (ResidualUnit  (None, 56, 56, 64)       74240     
 )                                                               
                                                        